In [1]:
import requests
import pandas as pd
import tqdm
import requests
import gzip
import csv
from wikidata.client import Client


def load_qrank(path):    
    with gzip.open(path, mode='rt') as f:
        reader = csv.reader(f)
        next(reader)
        qid_to_qrank = {}
        for qid, qrank in tqdm.tqdm(reader):
            qid_to_qrank[qid] = int(qrank)
    return qid_to_qrank


def query_wikidata(
    sparql_query,
    endpoint_url="https://query.wikidata.org/sparql?format=json"
):
    r = requests.get(endpoint_url, params={"query": sparql_query}).json()
    return r


def wikidata_id_to_label(qid):
    client = Client()
    entity = client.get(qid, load=True)
    data = entity.data    
    label = data["labels"]["en"]["value"]
    return label


def sparql_to_df(
    sparql_query,
    result_post_processors,
    endpoint="https://query.wikidata.org/sparql?format=json",    
):
    res = requests.get(endpoint, params={"query": sparql_query}).json()
    rows = []
    for r in res['results']['bindings']:
        row = {}
        for k, v in r.items():
            postprocess = result_post_processors.get(k, lambda x: x)
            v = postprocess(v['value'])
            row[k] = v
        rows.append(row)    
    return pd.DataFrame.from_records(rows)

### Load QRank entity list

**Warning:** this takes 2-3 GB RAM. <br>
This list will help us to rank lists of entities so that we can select useful top-k example datasets.<br>
The ranking is based on page view counts from Wikipedia and Wiki-related pages. <br>
More info here: https://github.com/brawer/wikidata-qrank/tree/main

In [2]:
!wget -nc "https://qrank.wmcloud.org/download/qrank.csv.gz"

File ‘qrank.csv.gz’ already there; not retrieving.



In [3]:
qid_to_qrank = load_qrank("qrank.csv.gz")

26226364it [00:24, 1075520.94it/s]


# S&P 500

First we get a large list of companies listed in common exchanges. We store the Wikidata ID, Wikidata label and ticker of these companies.

In [4]:
companies_query = """SELECT DISTINCT ?Wikidata_ID ?Wikidata_Label ?Ticker
WHERE {
    ?Wikidata_ID wdt:P31/wdt:P279* wd:Q4830453 . # entity must be a business
    ?Wikidata_ID p:P414 ?exchange . # entity must have an exchange;
    { ?exchange ps:P414 wd:Q13677 } # NYSE
      UNION { ?exchange ps:P414 wd:Q82059 } # NASDAQ
      UNION { ?exchange ps:P414 wd:Q1071853 }. # OR CBOE
    ?exchange pq:P249 ?Ticker . # entity must have ticker in exchange
    ?Wikidata_ID rdfs:label ?Wikidata_Label. FILTER( LANG(?Wikidata_Label)="en" )
}"""
companies_df = sparql_to_df(
    companies_query,
    result_post_processors={
        'Wikidata_ID': lambda uri: uri.replace("http://www.wikidata.org/entity/", ""),
    }
)
companies_df

,Wikidata_ID,Ticker,Wikidata_Label
0,Q63335,FL,Foot Locker
1,Q54173,GE,General Electric
2,Q58024,EBAY,eBay
3,Q63327,ABNB,Airbnb
4,Q122141,VOD,Vodafone
...,...,...,...
3126,Q112659289,ASTS,AST SpaceMobile
3127,Q113153195,DIDI,DiDi Global
3128,Q115258245,DSP,Viant Technology
3129,Q115446257,SEER,"Seer, Inc"


Next, we fetch a list of S&P 500 companies from a table on the S&P 500 Wikipedia page.

In [5]:
smp_wiki_tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
smp_wiki_table = smp_wiki_tables[0]
smp_wiki_table = smp_wiki_table.rename(columns={"Symbol": "Ticker"})

Now we can find the S&P-500 companies in our big company list based on their tickers. <br>
We also do another few processing steps:
- deduplicate entries based on Wikidata IDs and stock tickers
- only select the top-100 companies using QRank

In [6]:
# merge to get Wikidata ID for each entry from Wikipedia S&P-500 table
smp_df = pd.merge(left=smp_wiki_table, right=companies_df)

# rank entities using QRank
smp_df = smp_df.sort_values(
    by="Wikidata_ID",
    key=lambda qids: [qid_to_qrank.get(qid) or 0 for qid in qids],
    ascending=False
)

# drop rows with duplicated Wikidata IDs, keeping higher-ranked items
smp_df = smp_df.drop_duplicates("Wikidata_ID", keep="first")
smp_df = smp_df.drop_duplicates("Ticker", keep="first")

# Final CSV only contains English Wikidata label and Wikidata ID
smp_df = smp_df[["Wikidata_Label", "Wikidata_ID"]]
smp_df = smp_df.rename(columns={"Wikidata_Label": "Wikidata Label", "Wikidata_ID": "Wikidata ID"})
smp_df = smp_df.reset_index(drop=True)
smp_df = smp_df[:100]
smp_df

,Wikidata Label,Wikidata ID
0,Google,Q95
1,Netflix,Q907311
2,Meta Platforms,Q380
3,Amazon,Q3884
4,Apple,Q312
...,...,...
95,Cummins,Q173476
96,Schlumberger,Q1425316
97,Capital One,Q1034654
98,Eli Lilly and Company,Q632240


In [7]:
smp_df.to_csv("smp500.csv", index=False)

# Nasdaq-100

We gather the Nasdaq-100 companies pretty much in the same way as above, using the same company list with IDs and labels. <br>
We just apply it to a different table downloaded from Wikipedia. <br>
We skip the ranking step since we're happy with k=100

In [8]:
nasdaq_wiki_tables = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")
nasdaq_df_table = nasdaq_wiki_tables[4]

In [9]:
nasdaq_wiki_tables = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")
nasdaq_df_table = nasdaq_wiki_tables[4]

# merge to get Wikidata ID for each entry from Wikipedia S&P-500 table
nasdaq_df = pd.merge(left=nasdaq_df_table, right=companies_df)

# rank entities using QRank
nasdaq_df = nasdaq_df.sort_values(
    by="Wikidata_ID",
    key=lambda qids: [qid_to_qrank.get(qid) or 0 for qid in qids],
    ascending=False
)

# drop rows with duplicated Wikidata IDs
nasdaq_df = nasdaq_df.drop_duplicates("Wikidata_ID", keep="first")
nasdaq_df = nasdaq_df.drop_duplicates("Ticker", keep="first")

# Final CSV only contains English Wikidata label and Wikidata ID
nasdaq_df = nasdaq_df[["Wikidata_Label", "Wikidata_ID"]]
nasdaq_df = nasdaq_df.rename(columns={"Wikidata_Label": "Wikidata Label", "Wikidata_ID": "Wikidata ID"})
nasdaq_df = nasdaq_df.reset_index(drop=True)
nasdaq_df = nasdaq_df[:100]
nasdaq_df

,Wikidata Label,Wikidata ID
0,Google,Q95
1,Netflix,Q907311
2,Meta Platforms,Q380
3,Amazon,Q3884
4,Apple,Q312
...,...,...
94,CoStar Group,Q16988748
95,Verisk Analytics,Q7921370
96,"IDEXX Laboratories, Inc.",Q1758392
97,Old Dominion Freight Line,Q7083862


**Note**: We end up with 99 instead of 100 entries because Google appears twice in the list due to its separate GOOG and GOOGL tickers.

In [10]:
smp_df.to_csv("nasdaq100.csv", index=False)

# US-Politicians

We directly get this list using Wikidata attributes, no need for parsing Wikipedia tables.

In [11]:
politicians_query = """
SELECT DISTINCT ?politician ?label ?aliases ?desc
WHERE {
  ?politician wdt:P31 wd:Q5; # Human
              wdt:P106 wd:Q82955; # Occupation: Politician
              wdt:P27 wd:Q30; # Country of citizenship: United States
              wdt:P39 ?position. # Position held

  # National-level positions
  VALUES ?position {
    wd:Q11696 # President of the United States
    wd:Q11699 # Vice President of the United States
    wd:Q13218630 # Member of the United States House of Representatives
    wd:Q4416090 # United States Senator
  }

  FILTER NOT EXISTS { ?politician wdt:P570 ?deathDate. } # Filter out politicians with a death date

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en".
      ?politician rdfs:label ?label.
      ?politician skos:altLabel ?aliases.
      ?politician schema:description ?desc.
  }
}
ORDER BY ?label
"""

politicians_df = sparql_to_df(
    politicians_query,
    result_post_processors={
        'politician': lambda uri: uri.replace("http://www.wikidata.org/entity/", ""),
    }
)
politicians_df

,politician,label,aliases,desc
0,Q302659,Aaron Schock,"Aaron Jon Schock, Aaron Shock",American politician
1,Q24435337,Abby Finkenauer,Abby Lea Finkenauer,American politician
2,Q55603085,Abigail Spanberger,Abigail Anne Spanberger,American politician
3,Q349955,Adam Kinzinger,Adam Daniel Kinzinger,American politician
4,Q350663,Adam Putnam,Adam Hughes Putnam,American politician
...,...,...,...,...
1594,Q538206,Yvonne Brathwaite Burke,NaN,American politician
1595,Q84719792,Zach Nunn,NaN,American politician
1596,Q139389,Zach Wamp,Zachary Paul Wamp,American politician
1597,Q139887,Zack Space,NaN,American politician


In [12]:
# rank politicians using QRank
politicians_df = politicians_df.sort_values(
    by="politician",
    key=lambda qids: [qid_to_qrank.get(qid) or 0 for qid in qids],
    ascending=False
)

# drop rows with duplicated Wikidata IDs
politicians_df = politicians_df.drop_duplicates("politician", keep="first")

# Final CSV only contains English Wikidata label and Wikidata ID
politicians_df = politicians_df[["label", "politician"]]
politicians_df = politicians_df.rename(columns={"label": "Wikidata Label", "politician": "Wikidata ID"})
politicians_df = politicians_df.reset_index(drop=True)
politicians_df = politicians_df[:100]
politicians_df

,Wikidata Label,Wikidata ID
0,Joe Biden,Q6279
1,Donald Trump,Q22686
2,Barack Obama,Q76
3,Nancy Pelosi,Q170581
4,Jimmy Carter,Q23685
...,...,...
95,Tom Cotton,Q3090307
96,John Edwards,Q217314
97,Patty Murray,Q258825
98,Joe Kennedy III,Q1707784


In [13]:
politicians_df.to_csv("us-politicians.csv", index=False)